# Check devices type

In [ ]:
import tensorflow as tf
from tensorflow.python.client import device_lib

tf.test.gpu_device_name()
device_lib.list_local_devices()

# Load and reform the data

In [ ]:
import json
import numpy as np
from sklearn.model_selection import train_test_split

json_path = "/content/drive/MyDrive/extracted_data.json"

with open(json_path, "r") as f:
    data = json.load(f)

# Make numpy arrays from dictionary
# 3D arrays (song, sample ,MFCC)
X = np.array(data["mfcc"])
y = np.array(data["labels"])

# Split data (train=80%, test=20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0, stratify=y)
# Split data (train=60%, validation=20%)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=0)


input_shape = X_train[0].shape

# Build and save model

In [ ]:
import keras
from keras import regularizers
from tensorflow.keras.layers import LSTM, Dense, Dropout
from keras.models import load_model

model = keras.Sequential()

# Input Layer
model.add(LSTM(128,input_shape=input_shape))
model.add(Dropout(0.2))

# 1st Hidden Layer
model.add(Dense(128, activation='relu'))

# 2nd Hidden Layer
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.4))

# 3rd Hidden Layer
model.add(Dense(48, activation='relu'))
model.add(Dropout(0.4))

# Output Layer
model.add(Dense(24, activation='softmax'))

model.summary()

model.compile(optimizer='adam', loss='SparseCategoricalCrossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=150, batch_size=20, validation_data=(X_val, y_val), shuffle=False)
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print('\nTest accuracy:', test_acc)

model_name = 'Genre_model.h5'
model.save(model_name)

# Visualizing the results

In [ ]:
from matplotlib import pyplot as plt

# Plot train and validation set accuracy
plt.figure(0)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Accuracy Plot')
plt.legend()

# Plot train and validation set error
plt.figure(1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Error')
plt.title('Error Plot')
plt.legend()